In [28]:
import pandas as pd
from sqlalchemy import create_engine

from flask import request, Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from io import TextIOWrapper
import csv
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.decomposition import LatentDirichletAllocation # Latent Dirichlet Allocation is a topic model that is used for discovering abstract topics from a collection of documents (variational Bayes algorithm)
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np

engine = create_engine('sqlite://', echo=False)

In [81]:
long_data = pd.read_csv('../raw_data/adr_labelled_data.csv', nrows = 299)
long_data = long_data.drop(columns = 'Unnamed: 8')
long_data_full = long_data

In [82]:
long_data = long_data[long_data['rating'] < 5]
long_data = long_data.reset_index(drop = True)

In [69]:
long_data_full.tail()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sideEffect
294,11521,Pramipexole,Restless Legs Syndrome,"""This has really helped me. I have had RLS all...",9,11-Mar-10,28,1
295,43990,Ethinyl estradiol / norgestimate,Birth Control,"""First of all, I started taking birth control ...",1,3-Jun-12,24,1
296,174896,AndroGel,"Hypogonadism, Male","""I am 69 years old. Prior to starting 5 pump...",5,7-Feb-11,41,1
297,85483,Ethinyl estradiol / norgestimate,Birth Control,"""Face cleared. Gained 5 pounds. No vomiting fo...",7,20-Jan-15,7,1
298,178737,Victoza,"Diabetes, Type 2","""All the side effects""",1,26-Dec-15,5,1


In [62]:
long_data.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sideEffect
0,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,1
1,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5,1
2,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1,9-Aug-16,11,1
3,191290,Pentasa,Crohn's Disease,"""I had Crohn&#039;s with a resection 30 years ...",4,6-Jul-13,8,1
4,221320,Dextromethorphan,Cough,"""Have a little bit of a lingering cough from a...",4,7-Sep-17,1,1


In [57]:
import string # Collection of string operations
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer #Lemmatize using WordNet's built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet.
from nltk import word_tokenize

from nltk.sentiment.util import mark_negation

import nltk

from flask import request, Flask, render_template
from flask_sqlalchemy import SQLAlchemy
import csv


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


app = Flask(__name__)


"""
Functions
"""

stop_words = set(stopwords.words('english'))

NEGATIONS = ["no", "not", "shouldn't", "aren't", "couldn't", "didn't", "doesn't", "don't", "wasn't", "weren't", "wouldn't", "nor"]

NEW_WORDS = ['mg', "month", "year", "day", "week", "time", "im", "ive", "hour"]

for negation in NEGATIONS:
    stop_words.remove(negation)

for new_word in NEW_WORDS:
    stop_words.add(new_word)

def to_list(x):
    list_words = x.split(' ')
    return list_words

def to_string(x):
    string = " ".join(x)
    return string

#===============================================================

def punctuation(x):
    for punctuation in string.punctuation:
        x =  x.replace(punctuation, '')
    return x.lower()

def remove_numbers (x):
    words_only = ''.join([i for i in x if not i.isdigit()])
    return words_only

def m_negation(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    tokenized_neg = mark_negation(without_stopwords)
    return tokenized_neg

def remove_stopwords(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

def lemmatize_review(x):
    lemma = WordNetLemmatizer()
    lista = []
    for w in x:
       lista.append(lemma.lemmatize(w))
    return lista

#===============================================================

def count_words(x):
    wordfreq = []
    for w in x:
        wordfreq.append(x.count(w))
    return dict(zip(x, wordfreq))

def total_count(x):
    total_count = {}
    for row in x:
        for key in row.keys():
          if key in total_count:
              total_count[key] += 1
          else:
              total_count[key] = 1
    return pd.DataFrame(sorted(total_count.items(), key=lambda x: x[1], reverse=True)).head(30).T

def round_two(x):
    return str(int(round(x, 2) * 100)) + "%"

def one_or_zero(x):
    # Makes the prediction a binary outpur
    if x <= 0.5:
        x = 1
    else:
        x = 0
    return x

#===============================================================

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], round(topic[i], 2))
                        for i in topic.argsort()[:-10 - 1:-1]])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pefle\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pefle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pefle\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [75]:
manual = long_data.drop(["uniqueID", "drugName", "condition", "date", "rating", "usefulCount"], axis = 1)

manual["clean_review"] = manual["review"].apply(punctuation)
manual['clean_review'] = manual.clean_review.apply(remove_numbers)
manual['clean_review_lst'] = manual.clean_review.apply(to_list)

manual["NonStopwords_review_lst"] = manual.clean_review.apply(remove_stopwords)
manual["NonStopwords_review_str"] = manual.NonStopwords_review_lst.apply(to_string)

manual["NonStopwords_review_lst_MN"] = manual.clean_review.apply(m_negation)
manual["NonStopwords_review_str_MN"] = manual.NonStopwords_review_lst_MN.apply(to_string)

manual["Lemmatized_review_lst"] = manual.NonStopwords_review_lst_MN.apply(lemmatize_review)
manual["Lemmatized_review_str"] = manual.Lemmatized_review_lst.apply(to_string)

manual["Lemmatized_review_list"] = manual.NonStopwords_review_lst.apply(lemmatize_review)
manual["Lemmatized_review"] = manual.Lemmatized_review_list.apply(to_string)
manual["Lemmatized_review_list"] = manual.Lemmatized_review.apply(remove_stopwords)
manual["Lemmatized_review"] = manual.Lemmatized_review_list.apply(to_string)

# manaul['filered_column_by_se'] = 'juan.jack'
# manual['seperating by reviews_length'] = 'hendrike'
# manual['seperating by reviews_score'] = 'peter'

manuallong = long_data_full.drop(["uniqueID", "drugName", "condition", "date", "rating", "usefulCount"], axis = 1)


manuallong["clean_review"] = manuallong["review"].apply(punctuation)
manuallong['clean_review'] = manuallong.clean_review.apply(remove_numbers)
manuallong['clean_review_lst'] = manuallong.clean_review.apply(to_list)


manuallong["NonStopwords_review_lst"] = manuallong.clean_review.apply(remove_stopwords)
manuallong["NonStopwords_review_str"] = manuallong.NonStopwords_review_lst.apply(to_string)


manuallong["NonStopwords_review_lst_MN"] = manuallong.clean_review.apply(m_negation)
manuallong["NonStopwords_review_str_MN"] = manuallong.NonStopwords_review_lst_MN.apply(to_string)


manuallong["Lemmatized_review_lst"] = manuallong.NonStopwords_review_lst_MN.apply(lemmatize_review)
manuallong["Lemmatized_review_str"] = manuallong.Lemmatized_review_lst.apply(to_string)


manuallong["Lemmatized_review_list"] = manuallong.NonStopwords_review_lst.apply(lemmatize_review)
manuallong["Lemmatized_review"] = manuallong.Lemmatized_review_list.apply(to_string)
manuallong["Lemmatized_review_list"] = manuallong.Lemmatized_review.apply(remove_stopwords)
manuallong["Lemmatized_review"] = manuallong.Lemmatized_review_list.apply(to_string)





manual["words_count"] = manual.Lemmatized_review_list.apply(count_words)

X = manual["Lemmatized_review_str"]

y = manual["sideEffect"]

# # How many drugs the DF mentions
# drugs = pd.DataFrame(manual["drugName"].value_counts()).head(14).T
# drugs.to_csv('drug_mentions.csv')

# # How many conditions the DF mentions
# conditions = pd.DataFrame(manual["condition"].value_counts()).head(14).T
# conditions.to_csv('condition_mentions.csv')

manual = manual.drop(["clean_review", "clean_review_lst", "NonStopwords_review_lst", "NonStopwords_review_str"], axis = 1)

"""
TF-IDF features | Latent Dirichlet allocation
"""

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 1000,
                             max_features = None,
                             vocabulary = None,
                             ngram_range = (1, 3)).fit(manual["Lemmatized_review"]) #

# MINDF Ignore terms that have a document frequency strictly higher than the given threshold
# MAXDF When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold

data_vectorized = vectorizer.transform(manual["Lemmatized_review"]) #

lda_model = LatentDirichletAllocation(n_components = 2,
                                      learning_method = 'online',
                                      random_state = 29,
                                      #batch_size = 128,
                                      learning_decay = 0.5,
                                      learning_offset = 5,
                                      #evaluate_every = -1,
                                      verbose = 0,
                                      max_iter = 50).fit(data_vectorized)

vocab = vectorizer.get_feature_names()

# iterates over the reviews and predict
predictions = []
for review in manuallong["Lemmatized_review"]:
    vectorized = vectorizer.transform([review])
    lda_vectors = predictions.append(lda_model.transform(vectorized))

predictions = np.concatenate(predictions, axis=0)

# comparison DataFrame

compare_data = pd.DataFrame(predictions, columns = ["Side_Effect", "No_Side_Effect"])

length = len(predictions)

"""
Output Engineering
"""
print(manual['sideEffect'])
compare_data["Manually_Labelled"] = manuallong["sideEffect"] # Brings a column from the other DataFrame
print(compare_data['Manually_Labelled'])

compare_data["Prediction"] = compare_data.Side_Effect.apply(one_or_zero) # Applies the binary output
compare_data["No_Side_Effect_%"] = compare_data.No_Side_Effect.apply(round_two) # Applies a format
compare_data["Side_Effect_%"] = compare_data.Side_Effect.apply(round_two) # Applies a format

compare_data["bool"] = np.where(compare_data["Manually_Labelled"] == compare_data["Prediction"], True, False) # Compares betwen the precdiction and the label

compare_data.drop(["Side_Effect", "No_Side_Effect"], axis = 1)

0     1
1     1
2     1
3     1
4     1
     ..
73    1
74    1
75    1
76    1
77    1
Name: sideEffect, Length: 78, dtype: int64
0      0
1      1
2      1
3      1
4      1
      ..
294    1
295    1
296    1
297    1
298    1
Name: Manually_Labelled, Length: 299, dtype: int64


,Manually_Labelled,Prediction,No_Side_Effect_%,Side_Effect_%,bool
0,0,1,84%,16%,False
1,1,1,85%,15%,True
2,1,1,87%,13%,True
3,1,1,86%,14%,True
4,1,1,57%,42%,True
...,...,...,...,...,...
294,1,1,82%,18%,True
295,1,1,90%,10%,True
296,1,1,85%,15%,True
297,1,1,86%,14%,True


In [76]:
compare_data['bool'].value_counts()

True     167
False    132
Name: bool, dtype: int64

In [79]:
167/299

0.5585284280936454

In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

long_data['vectorized_review'] = data_vectorized

In [89]:
X = long_data[['vectorized_review', 'rating']]
y = long_data['sideEffect']

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)

In [93]:
X_train

,vectorized_review,rating
40,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",4
16,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",2
42,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",1
47,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",3
50,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",3
7,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",1
28,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",1
74,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",4
45,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",1
69,"(0, 138)\t0.193145989541962\n (0, 137)\t0.2...",1


In [92]:
logr = LogisticRegression()
logr.fit(X_train, y_train)
logr.score(X_test, y_test)

ValueError: setting an array element with a sequence.

In [21]:
data = {'drug' : ['Tylenol', 'Cialis', 'Ibuprofen', 'Benadril', 'Claritin'],
        'condition' : ['pain', 'ed', 'pain', 'allergy', 'allergy'],
       'headache' : [2, 4, 0, 1, 0],
       'nausea' : [0, 1, 1, 1, 0],
       'dizzy' : [0, 2, 1, 1, 0],
       'death' : [1, 0, 1, 2, 20]}

df = pd.DataFrame(data, columns = data.keys())

df = df.set_index('drug')

In [22]:
df.to_sql('reactions', con=engine, if_exists = 'replace')

In [23]:
engine.execute("SELECT * FROM reactions").fetchall()

[('Tylenol', 'pain', 2, 0, 0, 1),
 ('Cialis', 'ed', 4, 1, 2, 0),
 ('Ibuprofen', 'pain', 0, 1, 1, 1),
 ('Benadril', 'allergy', 1, 1, 1, 2),
 ('Claritin', 'allergy', 0, 0, 0, 20)]

In [29]:
print(df)

          condition  headache  nausea  dizzy  death
drug                                               
Tylenol        pain         2       0      0      1
Cialis           ed         4       1      2      0
Ibuprofen      pain         0       1      1      1
Benadril    allergy         1       1      1      2
Claritin    allergy         0       0      0     20


In [30]:
df.to_csv('../raw_data/dummydata.csv')

In [2]:
datatest = pd.read_csv('../raw_data/dummydata.csv')

In [4]:
html_block = pd.read_csv('../raw_data/dummydata.csv').to_html()

In [5]:
html_block

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>drug</th>\n      <th>condition</th>\n      <th>headache</th>\n      <th>nausea</th>\n      <th>dizzy</th>\n      <th>death</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Tylenol</td>\n      <td>pain</td>\n      <td>2</td>\n      <td>0</td>\n      <td>0</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Cialis</td>\n      <td>ed</td>\n      <td>4</td>\n      <td>1</td>\n      <td>2</td>\n      <td>0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Ibuprofen</td>\n      <td>pain</td>\n      <td>0</td>\n      <td>1</td>\n      <td>1</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Benadril</td>\n      <td>allergy</td>\n      <td>1</td>\n      <td>1</td>\n      <td>1</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Claritin</td>\n      <td>allergy</td>\n      <td>0</td>\n 

In [15]:
#render dataframe as html
html = df.to_html()

#write html to file
text_file = open("index.html", "w")
text_file.write(html)
text_file.close()

In [28]:
pd.read_sql_table('reactions', con = engine)

,drug,condition,headache,nausea,dizzy,death
0,Tylenol,pain,2,0,0,1
1,Cialis,ed,4,1,2,0
2,Ibuprofen,pain,0,1,1,1
3,Benadril,allergy,1,1,1,2
4,Claritin,allergy,0,0,0,20


In [32]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
db = SQLAlchemy(app)
csv_file = request.files['../raw_data/dummydata.csv']
csv_file = TextIOWrapper(csv_file, encoding='utf-8')
csv_reader = csv.reader('../raw_data/dummydata.csv', delimiter=',')
db.create_all()

class Drug(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    drugname = db.Column(db.String(30), unique=True)
#     condition = db.Column(db.String(20), unique=False)
#     headache = db.Column(db.Integer, unique=False)
#     nausea = db.Column(db.Integer, unique=False)
#     dizzy = db.Column(db.Integer, unique=False)
#     death = db.Column(db.Integer, unique=False)
    def __repr__(self):
        return '<Drug %r>' % self.drugname

# Drug.createall()
for row in csv_reader:
    print(row)
    drug = Drug(drugname=row[0])
    print(drug)
    db.session.add(drug)
    db.session.commit()
# fulldata = db.session.query(Drug).all()

c:\users\pefle\.venvs\adverse_drug_reactions\lib\site-packages\flask_sqlalchemy\__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.